### Курсовой проект

- Целевая метрика money precision@5. Порог для уcпешной сдачи проекта money precision @ 5 > 20%
- Есть бизнес-ограничения
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть ноутбук с решением + src + файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)


In [65]:
%matplotlib inline
%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k, money_precision_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
data = pd.read_csv('../course_files/retail_train.csv')
data_valid =  pd.read_csv('../course_files/retail_test1.csv')

In [94]:
data_filtered = prefilter_items(data, take_n_popular=2000)
data_filtered

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0,2.99
11,1364,26984896261,1,999999,1,2.19,31742,0.00,1520,1,0.0,0.0,2.19
12,1364,26984896261,1,999999,1,2.99,31742,-0.40,1520,1,0.0,0.0,2.99
13,1364,26984896261,1,999999,1,3.09,31742,0.00,1520,1,0.0,0.0,3.09
14,1364,26984896261,1,999999,1,2.50,31742,-0.99,1520,1,0.0,0.0,2.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2396794,1613,41655820646,663,999999,1,4.19,3262,0.00,1231,95,0.0,0.0,4.19
2396796,1613,41655820646,663,999999,1,4.99,3262,0.00,1231,95,0.0,0.0,4.99
2396797,1613,41655820646,663,999999,1,2.50,3262,-1.39,1231,95,0.0,0.0,2.50
2396798,1613,41655820646,663,999999,1,2.99,3262,-0.16,1231,95,0.0,0.0,2.99


In [99]:
recommender = MainRecommender(data_filtered, weighting=False)

In [100]:
mean_price = data_filtered['price'].mean()
bought_list = data_valid.groupby('user_id')['item_id'].unique().reset_index()
bought_list

,user_id,item_id
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109..."
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84..."
4,7,"[847270, 855557, 859987, 863407, 895454, 90663..."
...,...,...
1880,2496,"[829291, 862139, 912704, 933067, 933835, 95537..."
1881,2497,[6534178]
1882,2498,"[1053690, 1076875, 12386123, 858303, 920109, 1..."
1883,2499,"[826249, 895327, 9858944, 820321, 829291, 8323..."


In [101]:
mean_precision = []
for user, bought in zip(bought_list['user_id'], bought_list['item_id']):
    
    recommended_prices = []
    recommended_list = list(recommender.get_similar_items_recommendation(user))
    for item in recommended_list:
        price = data_filtered.loc[(data_filtered['item_id'] == item)]['price'].mean()
        recommended_prices.append(price)
    #print(recommended_list)
    #print(bought)
    
    
    recommended_prices = np.nan_to_num(recommended_prices,nan=0.1)
    #print(recommended_prices)
    item_precision = money_precision_at_k(recommended_list, bought, recommended_prices, 5)
    #print(item_precision)
    mean_precision.append(item_precision)
mean_precision = np.array(mean_precision)
mean_precision.mean()

0.04484834476529026

In [98]:
0.053547241758103103 - weigted=True, take_n_popular=5000
0.0550623896940243 - weigted=False, take_n_popular=5000
0.049024402364829564 - weigted=False, take_n_popular=10000
0.04484834476529026 - weigted=False, take_n_popular=2000
0.055596358258035744 - weigted=True, take_n_popular=2000

SyntaxError: can't assign to operator (<ipython-input-98-fd424bfb787e>, line 1)